In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installing CLIP

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.7 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-u4_8xeql
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-u4_8xeql
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369501 sha256=153f32995b7b3fe5e39f466a655339c765dac9582c7b1d8eddf7ed386f14c6dd
  Stored in directory: /tmp/pip-ephem-wheel-cache-kakiojdy/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
from PIL import Image
import torch
from torch import nn, optim
import glob
import os
import pandas as pd
import json
import numpy as np
import clip
from torch.utils.data import Dataset, DataLoader, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
from matplotlib.pyplot import imshow
import torchtext
import nltk, re, string, collections
from nltk.util import ngrams
import collections
%matplotlib inline
BATCH_SIZE = 128
EPOCH = 100

# Preparing Model and Data

In [ ]:
!unzip /content/drive/MyDrive/dataset/fashion-product-images-small.zip -d /content/fashion-product-images-small

Streaming output truncated to the last 5000 lines.
  inflating: /content/fashion-product-images-small/myntradataset/images/5813.jpg  
  inflating: /content/fashion-product-images-small/myntradataset/images/58131.jpg  
  inflating: /content/fashion-product-images-small/myntradataset/images/58132.jpg  
  inflating: /content/fashion-product-images-small/myntradataset/images/58133.jpg  
  inflating: /content/fashion-product-images-small/myntradataset/images/58135.jpg  
  inflating: /content/fashion-product-images-small/myntradataset/images/58136.jpg  
  inflating: /content/fashion-product-images-small/myntradataset/images/58137.jpg  
  inflating: /content/fashion-product-images-small/myntradataset/images/58138.jpg  
  inflating: /content/fashion-product-images-small/myntradataset/images/58139.jpg  
  inflating: /content/fashion-product-images-small/myntradataset/images/5814.jpg  
  inflating: /content/fashion-product-images-small/myntradataset/images/58140.jpg  
  inflating: /content/fashi

In [ ]:
df = pd.read_csv("/content/fashion-product-images-small/styles.csv",on_bad_lines='skip')
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [ ]:
IMG_ROOT = "/content/fashion-product-images-small/images"

In [ ]:
df['image_path'] = IMG_ROOT+'/'+df['id'].astype(str) + '.jpg'

In [ ]:
import os

# Define the image path
# image_path = CFG.image_path

# Function to check if the file exists
def file_exists(filename):
    file_path = os.path.join(filename)
    return os.path.exists(file_path)

# Apply the file_exists function to the 'image' column
df = df[df['image_path'].apply(file_exists)]

In [ ]:
df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image_path
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,/content/fashion-product-images-small/images/1...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,/content/fashion-product-images-small/images/3...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,/content/fashion-product-images-small/images/5...
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,/content/fashion-product-images-small/images/2...
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,/content/fashion-product-images-small/images/5...
...,...,...,...,...,...,...,...,...,...,...,...
44419,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe,/content/fashion-product-images-small/images/1...
44420,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop,/content/fashion-product-images-small/images/6...
44421,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt,/content/fashion-product-images-small/images/1...
44422,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume,/content/fashion-product-images-small/images/4...


In [ ]:
df['image_path']

0        /content/fashion-product-images-small/images/1...
1        /content/fashion-product-images-small/images/3...
2        /content/fashion-product-images-small/images/5...
3        /content/fashion-product-images-small/images/2...
4        /content/fashion-product-images-small/images/5...
                               ...                        
44419    /content/fashion-product-images-small/images/1...
44420    /content/fashion-product-images-small/images/6...
44421    /content/fashion-product-images-small/images/1...
44422    /content/fashion-product-images-small/images/4...
44423    /content/fashion-product-images-small/images/5...
Name: image_path, Length: 44419, dtype: object

In [ ]:
# IMG_ROOT = "/content/fashion-product-images-small/images"
img_paths = df['image_path'].tolist()

In [ ]:
len(img_paths)

44419

In [ ]:
d={}
for index, row in df.iterrows():
    image_path = row['image_path']
    value = ' '.join(str(row[column]) for column in df.columns if column not in ['id', 'year', 'image_path'])
    d[image_path] = value
len(d)

44419

## Splitting 20% for Validation

In [ ]:
train_img_paths, test_img_paths = train_test_split(img_paths, test_size=0.2, random_state=42)
d_train = {k: d[k] for k in train_img_paths}
d_test = {k: d[k] for k in test_img_paths}
len(d_train), len(d_test)

(35535, 8884)

## Loading Pre-trained CLIP Model and Preprocessor

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
model.load_state_dict(torch.load("/content/drive/MyDrive/Saved_Model/CLIP_Fashion_new_Epoch75.pt"))

image = preprocess(Image.open("/content/fashion-product-images-small/images/10000.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)
image.shape, text.shape

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 57.9MiB/s]


(torch.Size([1, 3, 224, 224]), torch.Size([3, 77]))

## FashionDataset

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image

class FashionDataset(Dataset):
    def __init__(self, data, preprocess):
        self.preprocess = preprocess
        self.img_paths = []
        self.captions = []
        self.path2label = {}

        for img_path, captions in data.items():
            self.img_paths.append(img_path)
            self.captions.append(captions)
            self.path2label[img_path] = len(self.path2label)

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = self.preprocess(Image.open(img_path))
        caption = self.captions[idx]
        label = self.path2label[img_path]
        return image, caption, label


train_dataset = FashionDataset(d_train, preprocess)
test_dataset = FashionDataset(d_test, preprocess)
len(train_dataset), len(test_dataset), train_dataset[0]


(35535,
 8884,
 (tensor([[[1.9303, 1.9303, 1.9303,  ..., 1.9303, 1.9303, 1.9303],
           [1.9303, 1.9303, 1.9303,  ..., 1.9303, 1.9303, 1.9303],
           [1.9303, 1.9303, 1.9303,  ..., 1.9303, 1.9303, 1.9303],
           ...,
           [1.9303, 1.9303, 1.9303,  ..., 1.9303, 1.9303, 1.9303],
           [1.9303, 1.9303, 1.9303,  ..., 1.9303, 1.9303, 1.9303],
           [1.9303, 1.9303, 1.9303,  ..., 1.9303, 1.9303, 1.9303]],
  
          [[2.0749, 2.0749, 2.0749,  ..., 2.0749, 2.0749, 2.0749],
           [2.0749, 2.0749, 2.0749,  ..., 2.0749, 2.0749, 2.0749],
           [2.0749, 2.0749, 2.0749,  ..., 2.0749, 2.0749, 2.0749],
           ...,
           [2.0749, 2.0749, 2.0749,  ..., 2.0749, 2.0749, 2.0749],
           [2.0749, 2.0749, 2.0749,  ..., 2.0749, 2.0749, 2.0749],
           [2.0749, 2.0749, 2.0749,  ..., 2.0749, 2.0749, 2.0749]],
  
          [[2.1459, 2.1459, 2.1459,  ..., 2.1459, 2.1459, 2.1459],
           [2.1459, 2.1459, 2.1459,  ..., 2.1459, 2.1459, 2.1459],
       

In [ ]:
i = 0
for k,v in train_dataset.path2label.items():
    i+=1
    print(k,v)
    if i == 10:
        break

/content/fashion-product-images-small/images/12590.jpg 0
/content/fashion-product-images-small/images/10356.jpg 1
/content/fashion-product-images-small/images/57475.jpg 2
/content/fashion-product-images-small/images/20528.jpg 3
/content/fashion-product-images-small/images/39124.jpg 4
/content/fashion-product-images-small/images/23727.jpg 5
/content/fashion-product-images-small/images/33663.jpg 6
/content/fashion-product-images-small/images/33250.jpg 7
/content/fashion-product-images-small/images/54736.jpg 8
/content/fashion-product-images-small/images/14703.jpg 9


## BalancedBatchSampler (ensures no same class per batch)

In [ ]:
# https://github.com/pytorch/pytorch/blob/e5742494f6080c8e6f43c37689fc18a7c4b39dfd/torch/utils/data/dataloader.py#L145
class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, labels, n_classes, n_samples):
        self.labels = labels
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.n_dataset = len(self.labels)
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < self.n_dataset:
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return self.n_dataset // self.batch_size

train_labels = torch.tensor([item[2] for item in train_dataset])
train_sampler = BalancedBatchSampler(train_labels, BATCH_SIZE, 1)
train_dataloader = DataLoader(train_dataset, batch_sampler=train_sampler)

test_labels = torch.tensor([item[2] for item in test_dataset])
test_sampler = BalancedBatchSampler(test_labels, BATCH_SIZE, 1)
test_dataloader = DataLoader(test_dataset, batch_sampler=test_sampler)



In [ ]:
for i, item in enumerate(train_sampler):
#     print(item)
#     print(len(item))
    labels = []
    for idx in item:
        label = train_dataset[idx][2]
        labels.append(label)
    break
len(labels), len(set(labels))

(128, 128)

In [ ]:
for batch in train_dataloader:
    imgs, txts, labels = batch
    print(imgs.shape)
    print(len(txts))
    print(labels)
    print(labels.shape)
    print(torch.unique(labels).shape)
    break

torch.Size([128, 3, 224, 224])
128
tensor([14765, 23697,  7404, 15659, 21396, 32820, 24064, 20910, 12378,  7372,
        16554,  5457, 14550, 33323,  4459, 12562, 10334, 18452, 11521, 30165,
         8600, 24858, 25946,  7155,   871,  3602,  5204,  2531, 20752, 16112,
        35370,  4127,  5486,   655, 27454, 29265, 17558,  2172, 20796,   602,
        25435,  9256,  2323, 17915,  7798, 11120,  9039,   585, 11739,  3177,
         3361, 21954, 10985, 13274, 22211, 28477, 34853,  2480,  8572,  5413,
        15772, 23549,  3018, 16352, 31007, 13190, 22045, 25656,  2824, 11760,
        34098, 16915, 27816, 32037, 26585,  7364, 18759, 20255,   800, 20485,
        25193, 13228, 15194, 33538, 28827, 15611,  6032, 17472, 30720, 28342,
        17756, 17601,  9088, 32744, 19391, 31882, 13391, 29963, 13894, 21497,
         5552,  9771, 27149,  9171, 11940, 30551, 35444, 27077,  4907, 17331,
         9876, 20054, 28795, 32219, 11965,  6579,  8447,  4568, 25219, 22180,
        12695,  5297,  7667, 

# Training

In [ ]:
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

if device == "cpu":
    model.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader)*EPOCH)

In [ ]:
best_te_loss = 1e5
best_ep = -1
for epoch in range(EPOCH):
    print(f"running epoch {epoch+1}, best test loss {best_te_loss} after epoch {best_ep}")
    step = 0
    tr_loss = 0
    model.train()
    pbar = tqdm(train_dataloader, leave=False)
    for batch in pbar:
        step += 1
        optimizer.zero_grad()

        images, texts, _ = batch
        images = images.to(device)
        texts = clip.tokenize(texts).to(device)
#         print(images.shape, texts.shape)
        logits_per_image, logits_per_text = model(images, texts)
        ground_truth = torch.arange(BATCH_SIZE).to(device)

        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        total_loss.backward()
        tr_loss += total_loss.item()
        if device == "cpu":
            optimizer.step()
            scheduler.step()
        else:
            convert_models_to_fp32(model)
            optimizer.step()
            scheduler.step()
            clip.model.convert_weights(model)
        pbar.set_description(f"train batchCE: {total_loss.item()}", refresh=True)
    tr_loss /= step

    step = 0
    te_loss = 0
    with torch.no_grad():
        model.eval()
        test_pbar = tqdm(test_dataloader, leave=False)
        for batch in test_pbar:
            step += 1
            images, texts, _ = batch
            images = images.to(device)
            texts = clip.tokenize(texts).to(device)
            logits_per_image, logits_per_text = model(images, texts)
            ground_truth = torch.arange(BATCH_SIZE).to(device)

            total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            te_loss += total_loss.item()
            test_pbar.set_description(f"test batchCE: {total_loss.item()}", refresh=True)
        te_loss /= step

    if te_loss < best_te_loss:
        best_te_loss = te_loss
        best_ep = epoch
    if ((epoch+1)%5==0):
      filename = f"/content/drive/MyDrive/Saved_Model/CLIP_Fashion_new_Epoch{epoch+1}.pt"
      torch.save(model.state_dict(), filename)
      print("Saved Best Model!!")
    print(f"epoch {epoch+1}, tr_loss {tr_loss}, te_loss {te_loss}")
torch.save(model.state_dict(), "/content/drive/MyDrive/Saved_Model/last_model.pt")

running epoch 76, best test loss 100000.0 after epoch -1


  0%|          | 0/277 [00:00<?, ?it/s]

  0%|          | 0/69 [00:00<?, ?it/s]

epoch 76, tr_loss 0.02529469638094575, te_loss 0.39049408401268115
running epoch 77, best test loss 0.39049408401268115 after epoch 0


  0%|          | 0/277 [00:00<?, ?it/s]

...

epoch 117, tr_loss 0.011512598406106557, te_loss 0.29700856968976447
running epoch 118, best test loss 0.29700856968976447 after epoch 41


  0%|          | 0/277 [00:00<?, ?it/s]

  0%|          | 0/69 [00:00<?, ?it/s]

epoch 118, tr_loss 0.012808027035062494, te_loss 0.30586132104846014
running epoch 119, best test loss 0.29700856968976447 after epoch 41


  0%|          | 0/277 [00:00<?, ?it/s]